In [ ]:
import os
import json
import re

def write_json(filename, maps):
    x = json.dumps(maps, indent=4)
    with open(filename, "w") as f:
        f.write(x)


def read_json(filename):
    json_cont = None
    with open(filename, encoding="ISO-8859-1") as f:
        json_cont = json.load(f)
    return json_cont


def make_dir(dirname):
    if os.path.exists(dirname):
        return
    os.makedirs(dirname)


# get all the files with some keyword
def list_keyword_in_filename(alist, keyword):
    res = []
    for x in alist:
        if keyword in x:
            res.append(x)
    return res


# find the symbol pairs, e.g "(->)", "{->}", "'->'"
def stack_match_symbol(content, left_idx, ls, rs):
    symbol = content[left_idx]
    list_parenthesis = [symbol]
    size = len(content)

    if ls == rs:
        for i in range(left_idx + 1, size):
            item = content[i]
            if item != ls:
                continue
            else:
                list_parenthesis.pop()
                if len(list_parenthesis) == 0:
                    return i
    else:
        for i in range(left_idx + 1, size):
            item = content[i]
            if item != ls and item != rs:
                continue
            if item == ls:
                list_parenthesis.append(ls)
            else:
                list_parenthesis.pop()
                if len(list_parenthesis) == 0:
                    return i
    return -1


# get the name, type, title from the string
def get_specific_items(
    input_str, name_pattern1_str, name_pattern2_str, name_pattern3_str, lead_str
):
    # lead_str == "'name': ", "'type': ", "'title': "
    name_str = ""
    tmp_name1 = re.search(name_pattern1_str, input_str)
    if tmp_name1 is None:
        tmp_name2 = re.search(name_pattern2_str, input_str)
        if tmp_name2 is None:
            if lead_str in input_str:
                tmp_name = re.search(name_pattern3_str, input_str)
                name_str = tmp_name[1].strip()
            else:
                return name_str
        else:
            left_pidx = tmp_name2.end() - 1
            right_pidx = stack_match_symbol(input_str, left_pidx, '"', '"')
            name_str = input_str[left_pidx + 1 : right_pidx]
    else:
        left_pidx = tmp_name1.end() - 1
        right_pidx = stack_match_symbol(input_str, left_pidx, "'", "'")
        name_str = input_str[left_pidx + 1 : right_pidx]

    return name_str

def get_method_detail(filename, vv):

    method_pattern = re.compile("public java.lang.Object " + vv + r"\(.*\) ({)")
    run = ""
    with open(filename, encoding="ISO-8859-1") as fr:
        content = fr.read()
        tmp_run = method_pattern.search(content)

        left_pidx = tmp_run.end() - 1
        right_pidx = stack_match_symbol(content, left_pidx, "{", "}")
        run = content[left_pidx + 1 : right_pidx]

    return run

def replace_action(lines, map_inputs):
    sendCommands = lambda name, capa, val: f"""
        context.api.devices.sendCommands(context.config.{name}, '{capa}', {val})
    """
    pattern_str = r"\.(\w+)\((.*)\)"
    res = ""
    for line in lines:
        for varname, info in map_inputs.items():
            try:
                line = line.strip()
                capa = info[0].replace("capability.","")
                ptr = f"{varname}{pattern_str}"
                list_ptr = re.search(ptr, line)
                if list_ptr is None:
                    continue
                xx = line.index("(")
                line = line[:xx]

                name, val = line.split(".")
                line=sendCommands(varname, capa, val)
                break
            except ValueError:
                pass
        res+="        "+line+"\n"
    return res


# store information from different files into one single file
def combine_info(map_info, name, action, inputs, subscribe, desc):
    map_info[name] = {}
    map_info[name]["name"] = desc[name]["name"]
    map_info[name]["description"] = desc[name]["description"]
    map_info[name]["input"] = inputs[name]
    map_info[name]["subscribe"] = subscribe[name]
    map_info[name]["action"] = action[name]


# join the list to str, used for compare if two app share the same inputs, subscribe, action
def get_info_str(name, action, inputs, subscribe, desc):
    str_name = desc[name]["name"]
    str_description = desc[name]["description"]
    map_inputs = inputs[name]
    map_subscribe = subscribe[name]
    list_action = action[name]

    str_inputs = "; ".join(
        sorted(
            [x for x in map_inputs.keys() if map_inputs[x][0].startswith("capability")]
        )
    )
    str_subscribe = "; ".join(
        sorted(map(lambda x: ", ".join(x), map_subscribe.get("subscribe", [])))
    )
    str_action = "; ".join(sorted(list_action))

    return str_inputs + str_subscribe + str_action

In [ ]:
test_folder = "Test2"
test_js_folder = "Test_JS"

script_pattern = re.compile(r"public class script extends groovy.lang.Script ({)")
pref_pattern = re.compile(r"preferences(\(.*\))?\s*{")
definition_pattern_str = r"this\.definition\((\[.*\])\)"

run_pattern = re.compile(r"public java.lang.Object run\(\) ({)")

description_pattern_str1 = r"'description':\s'(.*?)'[,|\]]"
description_pattern_str2 = r"'description':\s\"(.*?)\"[,|\]]"

name_pattern_str1 = r"'name':\s'(.*?)'[,|\]]"
name_pattern_str2 = r"'name':\s\"(.*?)\"[,|\]]"

section_pattern_str = r"this\.section(\()"
input_pattern_str = r"this\.input(\()"

name_pattern1_str = "'name':\s(')"
name_pattern2_str = "'name':\s(\")"
name_pattern3_str = "'name':\s(.*?),"

type_pattern1_str = "'type':\s(')"
type_pattern2_str = "'type':\s(\")"
type_pattern3_str = "'type':\s(.*?),"

title_pattern1_str = "'title':\s(')"
title_pattern2_str = "'title':\s(\")"
title_pattern3_str = "'title':\s(.*?)[,\]$]"

installed_pattern_str = r"public java.lang.Object installed\(\) ({)"
updated_pattern_str = r"public java.lang.Object updated\(\) ({)"
initialize_pattern_str = r"public java.lang.Object initialize\(\) ({)"
list_outter_pattern_str = [
    installed_pattern_str,
    updated_pattern_str,
    initialize_pattern_str,
]
# list_outter_pattern_str = [
#     updated_pattern_str,
# ]

subscribe_pattern_str = r"this.(subscribe(?:ToCommand)?)(\()"
schedule_pattern_str = r"this.(schedule)(\()"
run_pattern_str = r"this.(run\w{2,})(\()"
list_inner_patter_str = [subscribe_pattern_str, schedule_pattern_str, run_pattern_str]



jscode = lambda page_sections: """
const { SmartApp } = require('@smartthings/smartapp')

module.exports = new SmartApp()
    .enableEventLogging(2)
    .configureI18n()
    .page('mainPage', (context, page, configData) => {
""" + page_sections + """
    })
"""

page_sections = lambda section_name, section_list_str : """
        page.section('"""+section_name+"""', section => {
""" + section_list_str + """
        });

"""

updated = lambda detail: """
    .updated(async (context, updateData) => {
""" + detail + """
    })
"""

subscribe_to_devices = lambda name, capa, sub, handler : f"""
        await context.api.subscriptions.subscribeToDevices(context.config.{name}, '{capa}', '{sub}', '{handler}')
"""

subscribed_event_handler = lambda handler_name, detail: """
    .subscribedEventHandler('"""+ handler_name +"""', (context, event) => {
""" + detail + """
	})
"""

schedule = lambda name, handler_name, delay:f"""
        context.api.schedules.{name}('{handler_name}', delay);
"""

scheduled_event_handler = lambda handler_name, detail: """
    .scheduledEventHandler('"""+ handler_name +"""', (context, event) => {
""" + detail + """
	})
"""

sendCommands = lambda name, capa, val: f"""
        context.api.devices.sendCommands(context.config.{name}, '{capa}', {val})
"""


for fname in os.listdir(test_folder):

    print("=="*20+fname)
    filename = test_folder + "/" + fname
    filefolder = test_js_folder+"/"+fname
    make_dir(filefolder)
    locales_folder = filefolder +"/" + "locales" 
    make_dir(locales_folder)
    en_json = {}
    code = ""

    try:
        with open(filename, encoding="ISO-8859-1") as fr:
            

            content = fr.read()
            content = re.sub(r"script\d+", "script", content)
            tmp_script = script_pattern.search(content)

            left_pidx = tmp_script.end() - 1
            right_pidx = stack_match_symbol(content, left_pidx, "{", "}")
            script = content[left_pidx + 1 : right_pidx]

            # Separate run and remain
            tmp_run = run_pattern.search(script)

            left_pidx = tmp_run.end() - 1
            right_pidx = stack_match_symbol(script, left_pidx, "{", "}")
            # Run include preferences and definition
            run = script[left_pidx + 1 : right_pidx]
            # all the other functions, include installed, updated
            remain = script[: left_pidx - 30] + script[right_pidx + 1 :]

            # description

            tmp_def = re.findall(definition_pattern_str, run)
            tmp_def = list(set(tmp_def))

            desc, filenam2 = "", ""
            if len(tmp_def)>0:
                tmp_def = tmp_def[0]
                # Description
                tmp_desc1 = re.findall(description_pattern_str1, tmp_def)
                if len(tmp_desc1) == 0:
                    tmp_desc2 = re.findall(description_pattern_str2, tmp_def)
                    if len(tmp_desc2) == 0:
                        print(f"Error -> Has definition but no description {fname}")
                    else:
                        desc = tmp_desc2[0]
                else:
                    desc = tmp_desc1[0]

                # Filename
                tmp_name1 = re.findall(name_pattern_str1, tmp_def)
                if len(tmp_name1) == 0:
                    tmp_name2 = re.findall(name_pattern_str2, tmp_def)
                    if len(tmp_name2) == 0:
                        print(f"Error -> Has definition but no name {fname}")
                    else:
                        filename2 = tmp_name2[0]
                else:
                    filename2 = tmp_name1[0]


            # Section Input
            map_input = {}
            iter_tmp_section = re.finditer(section_pattern_str, run)
            section_js_str = ""
            for tmp_section in iter_tmp_section:
                left_pidx = tmp_section.end() - 1
                right_pidx = stack_match_symbol(run, left_pidx, "(", ")")
                section = run[left_pidx + 1 : right_pidx]
                section_str = ""
                # get section string
                sym_idx = section.find(", {")
                if section.strip()[0] == "{":
                    section_str = ""
                    section = section.strip()
                elif sym_idx == -1:
                    continue
                else:
                    section_str = section[:sym_idx]
                    section = section[sym_idx + 1 :]

                if "'title': " in section_str:
                    section_str = get_specific_items(
                        section_str,
                        title_pattern1_str,
                        title_pattern2_str,
                        title_pattern3_str,
                        "'title': ",
                    )

                # get input method
                tmp_input = re.finditer(input_pattern_str, section)
                input_js_str = ""
                for input_str in tmp_input:

                    left_pidx = input_str.end() - 1
                    right_pidx = stack_match_symbol(section, left_pidx, "(", ")")
                    input_str = section[left_pidx + 1 : right_pidx]

                    input_str = input_str.strip()

                    name_str, type_str, title_str = "", "", ""
                    if input_str[0] == "[" and input_str[-1] == "]":
                        # get name
                        name_str = get_specific_items(
                            input_str,
                            name_pattern1_str,
                            name_pattern2_str,
                            name_pattern3_str,
                            "'name': ",
                        )
                        # get type
                        type_str = get_specific_items(
                            input_str,
                            type_pattern1_str,
                            type_pattern2_str,
                            type_pattern3_str,
                            "'type': ",
                        )
                        # get title
                        title_str = get_specific_items(
                            input_str,
                            title_pattern1_str,
                            title_pattern2_str,
                            title_pattern3_str,
                            "'title': ",
                        )

                    elif input_str[0] != "[" and input_str[-1] != "]":
                        list_inputs = input_str.split(", ")
                        if len(list_inputs) == 2:
                            name_str, type_str = list_inputs
                        elif len(list_inputs) == 1:
                            # fset.add(fname)
                            name_str = list_inputs[0]
                        else:
                            pass
                    elif input_str[0] != "[" and input_str[-1] == "]":
                        pass
                    elif input_str[0] == "[" and input_str[-1] != "]":
                        left_pidx = 0
                        right_pidx = stack_match_symbol(input_str, left_pidx, "[", "]")
                        tmp_str = input_str[right_pidx + 2 :].strip().split(", ")
                        tmp_title = input_str[left_pidx + 1 : right_pidx]
                        if len(tmp_str) < 2:
                            if "'type': " in tmp_title:
                                type_str = get_specific_items(
                                    input_str,
                                    type_pattern1_str,
                                    type_pattern2_str,
                                    type_pattern3_str,
                                    "'type': ",
                                )
                            name_str = tmp_str[0]
                        else:
                            name_str, type_str = tmp_str[:2]

                        if "'title': " in tmp_title:
                            try:
                                title_str = get_specific_items(
                                    tmp_title,
                                    title_pattern1_str,
                                    title_pattern2_str,
                                    title_pattern3_str,
                                    "'title': ",
                                )
                            except TypeError as e:
                                tmp_name = re.search("'title':\s(.*?)$", tmp_title)
                                title_str = tmp_name[1].strip()
                    else:
                        pass

                    name_str = name_str.strip("'").strip('"')
                    type_str = type_str.strip("'").strip('"')
                    if type_str in ["capability.Switch", "capability.swicth"]:
                        type_str = "capability.switch"
                    elif type_str == "capability.EnergyMeter":
                        type_str = "capability.energyMeter"
                    elif type_str == "capability.phMeasurement":
                        type_str = "capability.pHMeasurement"
                    elif type_str == "capability.Polling":
                        type_str = "capability.polling"
                    elif type_str in [
                        "capability.liquidFlowRate",
                        "capability.pushableButton",
                        "capability.fanControl",
                        "capability.switchState",
                        "capability.nope",
                        "capability.voltageMeter",
                        "capability.airconditioner",
                        "capability.currentMeter",
                        "capability.hub",
                        "capability.type",
                    ]:
                        continue
                    elif type_str == "capability.temperature":
                        type_str = "capability.temperatureMeasurement"
                    elif type_str == "capability.Thermostat":
                        type_str = "capability.thermostat"
                    elif type_str == "capability.motion":
                        type_str = "capability.motionSensor"
                    elif type_str == "capability.Lock":
                        type_str = "capability.lock"
                    elif type_str == "capability.IlluminanceMeasurement":
                        type_str = "capability.illuminanceMeasurement"
                    elif type_str in ["capability.alarmSensor", "capability.Alarm"]:
                        type_str = "capability.alarm"
                    elif type_str == "capability.trackingMusicPlayer":
                        type_str = "capability.musicPlayer"
                    elif type_str == "capability.presence":
                        type_str = "capability.presenceSensor"
                    elif type_str == "capability.Sensor":
                        type_str = "capability.sensor"
                    elif type_str == "capability.Momentary":
                        type_str = "capability.momentary"
                    elif type_str == "capability.ContactSensor":
                        type_str = "capability.contactSensor"
                    # elif type_str == 'capability.type':
                    #     type_str = 'capability.contactSensor'

                    title_str = title_str.strip("'").strip('"')
                    section_str = section_str.strip("'").strip('"')
                    

                    if "capability." in type_str:
                        type_str = type_str.replace("capability.","")
                        input_js_str+=f"            section.deviceSetting('{name_str}').capability(['{type_str}']).name('{title_str}');\n"
                    elif type_str == "number":
                        input_js_str+=f"            section.numberSetting('{name_str}').name('{title_str}');\n"
                    elif type_str == "bool":
                        input_js_str+=f"            section.booleanSetting('{name_str}').name('{title_str}');\n"
                    elif type_str == "text":
                        input_js_str+=f"            section.textSetting('{name_str}').name('{title_str}');\n"
                    elif type_str == "enum":
                        input_js_str+=f"            section.enumSetting('{name_str}').name('{title_str}');\n"
                    elif type_str == "time":
                        input_js_str+=f"            section.timeSetting('{name_str}').name('{title_str}');\n"
                    map_input[name_str] = [type_str, title_str, section_str]
                section_js_str+=page_sections(section_str, input_js_str)

            code += jscode(section_js_str)
            
            # subscribe
            map_method = {}

            for out_pattern_str in list_outter_pattern_str:
                out_pattern = re.search(out_pattern_str, remain)

                if out_pattern is not None:
                    left_pidx = out_pattern.end() - 1
                    right_pidx = stack_match_symbol(remain, left_pidx, "{", "}")
                    out_str = remain[left_pidx + 1 : right_pidx]

                    for inner_pattern_str in list_inner_patter_str:
                        list_inner_pattern = re.finditer(inner_pattern_str, out_str)
                        for inner_pattern in list_inner_pattern:
                            method = inner_pattern[1]

                            if method not in map_method:
                                map_method[method] = set()

                            left_pidx = inner_pattern.end() - 1
                            right_pidx = stack_match_symbol(out_str, left_pidx, "(", ")")
                            method_detail = out_str[left_pidx + 1 : right_pidx]
                            map_method[method].add(method_detail)

        # https://stdavedemo.readthedocs.io/en/latest/smartapp-developers-guide/simple-event-handler-smartapps.html
        # subscribe(deviceName, "eventToSubscribeTo", handlerMethodName)
        # 1. Subscribe to All Device Events, e.g. "switch.on"
        # 2. Subscribe to All Device Events//Multiple Devices, e.g. "switch"
        # 3. Subscribe to Location Events, e.g. "mode", "position", "sunset", "sunrise", "sunsetTime", "sunriseTime"
        # shortcut for mode change handler
        # subscribe(location, modeChangeHandler)

        # https://stdavedemo.readthedocs.io/en/latest/smartapp-developers-guide/scheduling.html
        # schedule(cronExpression, handlerMethod)
        # schedule(dateTime, handlerMethod)
        # runIn(seconds, method, options)
        # runOnce(dateTime, handlerMethod, options)
        # runEvery5Minutes(handlerMethod)
        # runEvery10Minutes(handlerMethod)
        # runEvery15Minutes(handlerMethod)
        # runEvery30Minutes(handlerMethod)
        # runEvery1Hour(handlerMethod)
        # runEvery3Hours(handlerMethod)

        map_method_str = {}
        map_sub_sch = {"subscribedEventHandler":set(), "scheduledEventHandler":set()}
        update_str = ""
        update_set = set()
        for k, v in map_method.items():
            map_method_str[k] = []
            for it in v:
                tmp = list(map(lambda x: x.strip('"').strip("'"), it.split(", ")))

                if k in ["subscribe", "subscribeToCommand"]:
                    if len(tmp) == 2:
                        map_method_str[k].append(tmp)
                    elif len(tmp) >= 3:
                        map_method_str[k].append(tmp[:3])
                        if tmp[0]=='location':
                            update_set.add(subscribe_to_devices(tmp[0], 'location',tmp[1], tmp[2]))
                        elif tmp[0] in map_input:
                            update_set.add(subscribe_to_devices(tmp[0], map_input[tmp[0]][0],tmp[1], tmp[2]))
                        map_sub_sch["subscribedEventHandler"].add(tmp[2])
                    else:
                        pass
                elif k == "schedule":
                    if len(tmp) in [2]:
                        map_method_str[k].append(tmp)

                        update_set.add(schedule("schedule", tmp[1], tmp[0]))
                        map_sub_sch["scheduledEventHandler"].add(tmp[1])
                    else:
                        # pass
                        if tmp[1] == "doPoll":
                            map_method_str[k].append(tmp[:2])
                            update_set.add(schedule("schedule", tmp[1], tmp[0]))
                            map_sub_sch["scheduledEventHandler"].add(tmp[1])
                        else:
                            map_method_str[k].append([tmp[0] + ", " + tmp[1], tmp[2]])
                            update_set.add(schedule("schedule", tmp[2], tmp[0] + ", " + tmp[1]))
                            map_sub_sch["scheduledEventHandler"].add(tmp[2])

                elif k in ["runIn", "runOnce"]:
                    if len(tmp) >= 2:
                        map_method_str[k].append(tmp[:2])
                        update_set.add(schedule(k, tmp[1], tmp[0]))
                        map_sub_sch["scheduledEventHandler"].add(tmp[1])
                    else:
                        pass
                elif k in [
                    "runEvery5Minutes",
                    "runEvery10Minutes",
                    "runEvery15Minutes",
                    "runEvery30Minutes",
                    "runEvery1Hour",
                    "runEvery3Hours"]:
                    if len(tmp) >= 1:
                        map_method_str[k].append(tmp)
                        update_set.add(schedule(k, tmp[0], ''))
                        map_sub_sch["scheduledEventHandler"].add(tmp[0])
                    else:
                        pass
        for x in update_set:
            update_str+=x

        if len(map_method_str)>0:
            code += updated(update_str)

        for k, v in map_sub_sch.items():
            if len(v)==0:
                continue
            if k == "scheduledEventHandler":
                for vv in v:
                    run = get_method_detail(filename, vv)
                    lines =run.replace("evt.", "event.").replace("log.debug","console.log").replace("java.lang.Object","let").split("\n")
                    res = replace_action(lines, map_input)
                    code+=scheduled_event_handler(vv, res)
            elif k == "subscribedEventHandler":
                for vv in v:
                    run = get_method_detail(filename, vv)
                    lines =run.replace("evt.", "event.").replace("log.debug","console.log").replace("java.lang.Object","let").split("\n")
                    res = replace_action(lines, map_input)
                    code+=subscribed_event_handler(vv, res)
        print(code)


        
    except AttributeError:
        print(fname)

    write_json(locales_folder+"/"+"en.json", en_json)
    with open(filefolder+"/"+"smartapp.js","w") as fw:
        fw.write(code)